In [3]:
from transformers import (
    AutoImageProcessor,
    ResNetForImageClassification,
    GPT2LMHeadModel,
    AutoTokenizer,
    DataCollatorWithPadding,
)
import torch
from datasets import Dataset
from PIL import Image
from torch.utils.data import DataLoader, Subset
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import requests
from io import BytesIO


# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Load the CSV files into pandas DataFrames
df_image = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv')
df_report = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv')
# Create a DataFrame for images and captions
data = []
for i in range(len(df_image)):
    uid = df_image.iloc[i]['uid']
    image = df_image.iloc[i]['filename']
    index = df_report.loc[df_report['uid'] == uid]
    
    if not index.empty:    
        index = index.index[0]
        caption = df_report.iloc[index]['findings']
        if isinstance(caption, float):  # Skip rows with missing captions
            continue
        data.append({'imgs': image, 'captions': caption})

# Convert to a DataFrame
df = pd.DataFrame(data)
# Update image paths
loc = '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/'
df['imgs'] = loc + df['imgs']

# Convert pandas DataFrame to a Dataset object
dataset = Dataset.from_pandas(df)

# Loading pretrained models

In [5]:
# Load ResNet-50 for feature extraction (frozen)
resnet_model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50").to(device)
# Unfreeze the last layer of ResNet-50
for name, param in resnet_model.named_parameters():
    if "layer4" in name or "fc" in name:  # Fine-tune the last layer (layer4 and fully connected)
        param.requires_grad = True
    else:
        param.requires_grad = False

# Set ResNet-50 to train mode
resnet_model.train()


# Load GPT-2 for language generation
gpt2_model_name = "gpt2"  # or "distilgpt2" for a lighter version
tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
# GPT-2 doesn't have a pad token by default, let's assign one:
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
# Load ResNet-50 for feature extraction (frozen)
resnet_model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50").to(device)
for name, param in resnet_model.named_parameters():
    if "layer4" in name or "fc" in name:  # Fine-tune the last layer
        param.requires_grad = True
    else:
        param.requires_grad = False

resnet_model.train()  # Set ResNet-50 to train mode

# Load GPT-2 for language generation
gpt2_model_name = "gpt2"  # or "distilgpt2" for a lighter version
tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)
# GPT-2 doesn't have a pad token by default, let's assign one:
tokenizer.pad_token = tokenizer.eos_token

# Preprocessing data

In [10]:
# Processor for ResNet images
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
def preprocess_images_and_captions(example):
    # Process the image
    image = Image.open(example["imgs"]).convert("L")  # Convert to grayscale
    image = Image.merge("RGB", [image, image, image])  # Convert grayscale to RGB
    image_inputs = image_processor(image, return_tensors="pt")
    pixel_values = image_inputs["pixel_values"].squeeze(0)  # Shape [3, 224, 224]

    # Tokenize the caption
    text_inputs = tokenizer(
        example["captions"],
        truncation=True,
        max_length=32,  # Adjust as needed
        return_tensors="pt"
    )

    return {
        "pixel_values": pixel_values.tolist(),  # Convert tensor to list
        "input_ids": text_inputs["input_ids"].squeeze(0).tolist(),  # Convert tensor to list
        "attention_mask": text_inputs["attention_mask"].squeeze(0).tolist(),  # Convert tensor to list
    }

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

In [12]:
split = dataset.train_test_split(test_size=0.1, seed=42)  # 60% train, 40% test

training_dataset = split['train']
testing_dataset = split['test']

training_dataset = training_dataset.map(preprocess_images_and_captions)
testing_dataset = testing_dataset.map(preprocess_images_and_captions)

Map:   0%|          | 0/5822 [00:00<?, ? examples/s]

Map:   0%|          | 0/647 [00:00<?, ? examples/s]

In [14]:
text_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

def combined_collate_fn(batch):
    # Convert lists back to tensors
    pixel_values_list = [torch.tensor(item["pixel_values"]) for item in batch]
    input_ids_list = [torch.tensor(item["input_ids"]) for item in batch]
    attention_mask_list = [torch.tensor(item["attention_mask"]) for item in batch]

    # Debugging step
    for i, pv in enumerate(pixel_values_list):
        print(f"Sample {i}: type={type(pv)}, shape={pv.shape}")

    # Stack pixel values into a single tensor
    pixel_values = torch.stack(pixel_values_list, dim=0)  # [batch_size, 3, 224, 224]

    # Use Hugging Face DataCollatorWithPadding for tokenized text
    text_batch = {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
    }
    text_batch = text_collator(text_batch)

    # Add pixel values to the text batch
    text_batch["pixel_values"] = pixel_values
    return text_batch

dataloader = DataLoader(
    training_dataset,
    batch_size=10,         # only 2 samples in this example
    shuffle=True,
    collate_fn=combined_collate_fn,
    drop_last=True,      # can be True if you have many samples
)

In [15]:
class FeatureToCaption(nn.Module):
    """
    We:
      - Extract features from ResNet (outside this class, in the training loop, frozen)
      - Project them to GPT-2 hidden dim
      - Sum them with the GPT-2 token embeddings
    """
    def __init__(self, feature_dim=2048, hidden_dim=768, gpt2_name="gpt2"):
        super().__init__()
        self.linear = nn.Linear(feature_dim, hidden_dim)
        self.llm = GPT2LMHeadModel.from_pretrained(gpt2_name)
        # Because GPT-2 doesn't define pad_token by default
        self.llm.config.pad_token_id = tokenizer.eos_token_id

    def forward(self, resnet_features, input_ids, attention_mask):
        """
        resnet_features: [batch_size, feature_dim]
        input_ids:       [batch_size, seq_len]
        attention_mask:  [batch_size, seq_len]
        """
        # 1) Project the ResNet features to GPT-2 hidden size
        #    shape: [batch_size, hidden_dim]
        projected = self.linear(resnet_features)

        # 2) Expand them along seq_len dimension
        #    shape: [batch_size, 1, hidden_dim] -> [batch_size, seq_len, hidden_dim]
        batch_size, seq_len = input_ids.shape
        projected = projected.unsqueeze(1).expand(batch_size, seq_len, -1)

        # 3) GPT-2 token embeddings
        #    shape: [batch_size, seq_len, hidden_dim]
        token_embeds = self.llm.transformer.wte(input_ids)

        # 4) Sum them (the simplest approach)
        inputs_embeds = token_embeds + projected

        # 5) Forward pass through GPT-2
        outputs = self.llm(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            labels=input_ids,  # for CrossEntropyLoss
        )
        return outputs


In [16]:
# Instantiate our feature-to-caption model
model = FeatureToCaption(gpt2_name=gpt2_model_name).to(device)

################################################################################
# Training Loop (Minimal Example)
################################################################################
# Include ResNet-50 parameters in the optimizer
optimizer = optim.AdamW(
    list(model.parameters()) + list(resnet_model.parameters()),
    lr=1e-4
)


epochs = 3

for epoch in range(1, epochs + 1):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        # batch has "pixel_values", "input_ids", "attention_mask"
        pixel_values = batch["pixel_values"].to(device)      # [batch_size, 3, 224, 224]
        input_ids = batch["input_ids"].to(device)            # [batch_size, seq_len]
        attention_mask = batch["attention_mask"].to(device)  # [batch_size, seq_len]

        # Enable gradients for fine-tuning ResNet-50
        emb_out = resnet_model.resnet.embedder(pixel_values)
        enc_out = resnet_model.resnet.encoder(emb_out)
        pooled_features = resnet_model.resnet.pooler(enc_out.last_hidden_state).flatten(1)



        # Forward pass
        outputs = model(pooled_features, input_ids, attention_mask)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch}/{epochs}, Loss: {avg_loss:.4f}")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Sample 0: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 1: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 2: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 3: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 4: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 5: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 6: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 7: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 8: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 9: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 0: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 1: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 2: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample 3: type=<class 'torch.Tensor'>, shape=torch.Size([3, 224, 224])
Sample

In [10]:
#torch.save(model, "/kaggle/working/v2.pt")

# ---- Loading ----
#model = torch.load("/kaggle/working/v1.pt")
#model.eval()


## Testing the model

In [17]:
model.eval()

for sample in training_dataset:
    with torch.no_grad():
        # Convert pixel_values back to tensor
        pixel_values = torch.tensor(sample["pixel_values"]).unsqueeze(0).to(device)  # Add batch dimension
        #print(pixel_values.shape)  # Debugging
    
        # Convert input_ids and attention_mask to tensors
        input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0).to(device)
    
        # ResNet Features
        emb_out = resnet_model.resnet.embedder(pixel_values)
        enc_out = resnet_model.resnet.encoder(emb_out)
        pooled_features = resnet_model.resnet.pooler(enc_out.last_hidden_state).flatten(1)
    
        # Generate a caption
        outputs = model.llm.generate(
            inputs_embeds=(model.linear(pooled_features).unsqueeze(1) + model.llm.transformer.wte(input_ids)),
            attention_mask=attention_mask,
            max_length=100,
            num_beams=2,
        )
    
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated caption:", generated_text)
    print("Original caption:", sample["captions"])
    
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated caption:", generated_text)
    print("Original caption:", sample["captions"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption: 
Original caption: The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.
Generated caption: 
Original caption: The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.
Generated caption: 
Original caption: Lungs are clear. Heart is normal size. Trachea is midline. No pneumothorax. No large pleural effusion.
Generated caption: 
Original caption: Lungs are clear. Heart is normal size. Trachea is midline. No pneumothorax. No large pleural effusion.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption: iosis of the thoracic spine. Cardiomediastinal silhouette is normal in size and contour. No focal consolidation, pneumothorax or large pleural effusion.
Original caption: The lung volumes are low. However as compared to the prior study, there are increased perihilar opacities bilaterally. Stable dextroscoliosis of the lower thoracic spine with limited evaluation of the spinal XXXX. The tracheostomy tube is in unchanged position.
Generated caption: iosis of the thoracic spine. Cardiomediastinal silhouette is normal in size and contour. No focal consolidation, pneumothorax or large pleural effusion.
Original caption: The lung volumes are low. However as compared to the prior study, there are increased perihilar opacities bilaterally. Stable dextroscoliosis of the lower thoracic spine with limited evaluation of the spinal XXXX. The tracheostomy tube is in unchanged position.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption:  lobe granuloma unchanged. No focal area of consolidation, pneumothorax or large pleural effusion. Visualized osseous structures unremarkable.
Original caption: No acute osseous abnormality. The soft tissues are within normal limits. Normal appearing cardiomediastinal silhouette and hilar contours. Left lower lobe XXXX density XXXX representing atelectasis. No focal area of consolidation, pleural effusion, pneumothorax.
Generated caption:  lobe granuloma unchanged. No focal area of consolidation, pneumothorax or large pleural effusion. Visualized osseous structures unremarkable.
Original caption: No acute osseous abnormality. The soft tissues are within normal limits. Normal appearing cardiomediastinal silhouette and hilar contours. Left lower lobe XXXX density XXXX representing atelectasis. No focal area of consolidation, pleural effusion, pneumothorax.
Generated caption: .
Original caption: Lungs are clear without focal consolidation, effusion, or pneumothorax. Nor

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption:  adenopathy, adenopathy, adenopathy without focal consolidation, pneumothorax, effusion, or pneumothorax. Bony structures grossly intact.
Original caption: Heart size normal. Lungs are clear. XXXX are normal. No pneumonia, effusions, edema, pneumothorax, adenopathy, nodules or masses.
Generated caption:  adenopathy, adenopathy, adenopathy without focal consolidation, pneumothorax, effusion, or pneumothorax. Bony structures grossly intact.
Original caption: Heart size normal. Lungs are clear. XXXX are normal. No pneumonia, effusions, edema, pneumothorax, adenopathy, nodules or masses.
Generated caption: 
Original caption: The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.
Generated caption: 
Original caption: The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.


KeyboardInterrupt: 

# COMPUTE EVALUATION METRICS

## blue score

In [18]:
!pip install sacrebleu


from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import sacrebleu
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

model.eval()

smooth_fn = SmoothingFunction().method1  # A smoothing function to avoid zero scores on short sentences

for sample in training_dataset:
    with torch.no_grad():
        # Convert pixel_values back to tensor
        pixel_values = torch.tensor(sample["pixel_values"]).unsqueeze(0).to(device)
        
        # Convert input_ids and attention_mask to tensors
        input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0).to(device)
    
        # ResNet Features
        emb_out = resnet_model.resnet.embedder(pixel_values)
        enc_out = resnet_model.resnet.encoder(emb_out)
        pooled_features = resnet_model.resnet.pooler(enc_out.last_hidden_state).flatten(1)
    
        # Generate a caption
        outputs = model.llm.generate(
            inputs_embeds=(model.linear(pooled_features).unsqueeze(1) + model.llm.transformer.wte(input_ids)),
            attention_mask=attention_mask,
            max_length=100,
            num_beams=2,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Print for debugging
    print("Generated caption:", generated_text)
    print("Original caption:", sample["captions"])
    
    # ---- Compute BLEU for this single example ----
    # Assume 'sample["captions"]' is a string (single reference). 
    # If you have multiple references, see below for how to handle them.
    reference_str = sample["captions"]   # e.g. "A dog playing with a ball."
    hypothesis_str = generated_text      # e.g. "A dog is playing with a toy."

    # Tokenize
    reference_tokens = word_tokenize(reference_str.lower())   # or use a custom tokenizer
    hypothesis_tokens = word_tokenize(hypothesis_str.lower())
    
    # NLTK’s sentence_bleu expects a list of reference lists.
    # If you have exactly one reference per sample, do: [[reference_tokens]]
    score = sentence_bleu([reference_tokens], 
                          hypothesis_tokens, 
                          smoothing_function=smooth_fn)
    
    print("BLEU (sentence-level):", score)
    print("------------------------------------")


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.2 MB/s eta 0:00:00


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption: 
Original caption: The heart is normal in size. The mediastinum is unremarkable. The lungs are clear.
BLEU (sentence-level): 0
------------------------------------
Generated caption: 
Original caption: Lungs are clear. Heart is normal size. Trachea is midline. No pneumothorax. No large pleural effusion.
BLEU (sentence-level): 0
------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption: . No pneumothorax or pleural effusion.
Original caption: Stable position of right central venous catheter. Interval removal of nasogastric tube. Heart size is normal. Persistent prominent interstitial markings of the right upper lobe. There are no XXXX focal air space consolidations. No pleural effusions or pneumothoraces. The hilar and mediastinal contours are unchanged. Normal pulmonary vascularity.
BLEU (sentence-level): 8.009608821514565e-05
------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated caption:  rib fractures. Osseous structures intact.
Original caption: There are low volumes with bronchovascular crowding. No focal infiltrate or effusion. Heart and mediastinal contours within normal limits. No displaced fracture identified.
BLEU (sentence-level): 0.0022560374287911196
------------------------------------


KeyboardInterrupt: 

In [ ]:
#!pip install git+https://github.com/salaniz/pycocoevalcap.git
evalRefs = []
evalHyps = []

model.eval()
i=0
for idx, sample in enumerate(training_dataset):
    i+=1
    if i == 100:
        break
    with torch.no_grad():
        # Convert pixel_values back to tensor
        pixel_values = torch.tensor(sample["pixel_values"]).unsqueeze(0).to(device)
        
        # Convert input_ids and attention_mask to tensors
        input_ids = torch.tensor(sample["input_ids"]).unsqueeze(0).to(device)
        attention_mask = torch.tensor(sample["attention_mask"]).unsqueeze(0).to(device)
    
        # ResNet Features
        emb_out = resnet_model.resnet.embedder(pixel_values)
        enc_out = resnet_model.resnet.encoder(emb_out)
        pooled_features = resnet_model.resnet.pooler(enc_out.last_hidden_state).flatten(1)
    
        # Generate a caption
        outputs = model.llm.generate(
            inputs_embeds=(model.linear(pooled_features).unsqueeze(1) + model.llm.transformer.wte(input_ids)),
            attention_mask=attention_mask,
            max_length=100,
            #num_beams=2,
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Convert references into the required format
    # If sample["captions"] is a single string:
    if isinstance(sample["captions"], str):
        gt_captions = [sample["captions"]]
    else:
        gt_captions = sample["captions"]
    
    # Append references
    # pycocoevalcap expects something like:
    # {"image_id": <id>, "caption": "some reference caption"}
    for ref in gt_captions:
        evalRefs.append({
            "image_id": idx,
            "caption": ref
        })

    # Append hypothesis
    evalHyps.append({
        "image_id": idx,
        "caption": generated_text
    })

print(f"Number of references: {len(evalRefs)}")
print(f"Number of hypotheses: {len(evalHyps)}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [ ]:
from collections import defaultdict

from pycocoevalcap.cider.cider import Cider
gts = defaultdict(list)
res = {}

for ref in evalRefs:
    gts[ref["image_id"]].append(ref["caption"])

for hyp in evalHyps:
    res[hyp["image_id"]] = [hyp["caption"]]  # Wrap in a list to match the expected format

# Initialize CIDEr scorer
scorer = Cider()

# Compute CIDEr score
score, detailed_scores = scorer.compute_score(gts, res)

print("CIDEr Score:", score)  # Average CIDEr score
print("Detailed Scores:", detailed_scores)  # Per-sample scores

scorer = Cider()
score, detailed_scores = scorer.compute_score(evalRefs, evalHyps)
print("CIDEr Score:", score)


In [ ]:
!pip install pycocoevalcap